In [16]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import re
paths = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        paths.append(os.path.join(dirname, filename))
len(paths)
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

460

In [17]:
!pip install nibabel
import nibabel as nib

In [18]:
import matplotlib.pyplot as plt
from nilearn.masking import compute_brain_mask
import cv2 as cv
from scipy.ndimage import zoom

In [19]:
path_csv = '/kaggle/input/scan-labels/ADNI1_Complete_1Yr_1.5T_6_20_2025.csv'
labels = pd.read_csv(path_csv)

labels.head(30)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,Downloaded
0,I112538,941_S_1311,MCI,M,70,m12,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,6/01/2008,NiFTI,NaN
1,I97341,941_S_1311,MCI,M,70,m06,MRI,MPR-R; GradWarp; B1 Correction; N3; Scaled,Processed,9/27/2007,NiFTI,NaN
2,I97327,941_S_1311,MCI,M,69,sc,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,3/02/2007,NiFTI,NaN
3,I75150,941_S_1202,CN,M,78,m06,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,8/24/2007,NiFTI,NaN
4,I105437,941_S_1202,CN,M,79,m12,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,2/28/2008,NiFTI,NaN
5,I63874,941_S_1202,CN,M,78,sc,MRI,MPR-R; GradWarp; B1 Correction; N3; Scaled,Processed,1/30/2007,NiFTI,NaN
6,I79115,941_S_1197,CN,F,83,m06,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,8/30/2007,NiFTI,NaN
7,I66462,941_S_1197,CN,F,82,sc,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,1/20/2007,NiFTI,NaN
8,I108336,941_S_1197,CN,F,84,m12,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,5/21/2008,NiFTI,NaN
9,I63847,941_S_1194,CN,M,85,sc,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,1/20/2007,NiFTI,NaN


In [20]:
batch_size=8
batches_of_paths = [ paths[i:i + batch_size] for i in range(0, len(paths), batch_size) ]

#len(batches_of_paths)

In [21]:
def output_details(path):
    # 1. Extract Subject ID (safe indexing)
    parts = path.strip().split("/")
    subject_id = parts[6] if len(parts) > 6 else None

    # 2. Extract processing folder (safe)
    raw_folder = parts[7] if len(parts) > 7 else ""

    # 3. Normalize underscores: collapse multiple to single
    cleaned = re.sub(r'_+', '_', raw_folder).strip('_')  # e.g., 'MPR-R_N3_Scaled'
    components = cleaned.split("_")

    #print("Subject ID:", subject_id)
    #print("Components:", components)
        
    return subject_id, components

In [22]:
def create_string(components):
    string = ''
    for i, component in enumerate(components):
        string += component
        if i != len(components) - 1:
            string += '; '
    print(string)


subj_id, comp = output_details(paths[1])
print(comp)
create_string(comp)

['MPR-R', 'N3', 'Scaled']
MPR-R; N3; Scaled


In [23]:
def get_label(df, subj_id, description):
    match = df[(df['Subject'] == subj_id) & (df['Description'] == description)]
    
    if not match.empty:
        return match.iloc[0]['Group']
    else:
        return None

In [24]:
from tensorflow import keras
from tensorflow.keras import layers, models

# Hyperparameters
INPUT_SHAPE = (32, 32, 1)           # Change if your image shape is different
NUM_CLASSES = 3                    # Change if your number of classes is different
EPOCHS = 10
BATCH_SIZE = 32
reg_lambda = 0.00001

# Build LeNet model
def build_lenet():
    model = models.Sequential([
        layers.Conv2D(6, (5, 5), activation='relu', input_shape=INPUT_SHAPE, padding='same'),
        layers.AveragePooling2D(pool_size=(2, 2)),

        layers.Conv2D(16, (5, 5), activation='relu'),
        layers.AveragePooling2D(pool_size=(2, 2)),

        layers.Flatten(),
        layers.Dense(120, activation='relu'),
        layers.Dense(84, activation='relu'),
        layers.Dense(NUM_CLASSES, activation='softmax')  # For multi-class classification
    ])
    return model

# Create and compile model
model_lenet = build_lenet()
model_lenet.compile(optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
from skimage.transform import resize

total_batches = len(batches_of_paths)
train_batches = int(total_batches*0.8)
test_batches = total_batches-train_batches

## Training
batch_scans = []
batch_labels = []
for i in range(train_batches):
    batch = batches_of_paths[i]
    for path in batch:
        scan = nib.load(path)
        data = scan.get_fdata()
        data = resize(data, (256, 256, 180), anti_aliasing=False, preserve_range=True)
 
        details = output_details(path)
        subj_id = details[0]
        description = create_string(details[1])

        label = get_label(labels, subj_id, description)

        if label == 'MCI':
            continue
        elif label == 'AD':
            label = 1
        elif label == 'CN':
            label = 0

        batch_scans.append(data)
        batch_labels.append(label)

# Convert to arrays
batch_scans = np.array(batch_scans, dtype=np.float32)
batch_labels = np.array(batch_labels)

# Now you can feed into model like:
batch_slices = np.array([scan[:, :, scan.shape[2] // 2] for scan in batch_scans])  # shape: (8, 256, 256)

# Step 2: Add channel dimension for Conv2D input (grayscale)
batch_slices = batch_slices[..., np.newaxis]  # shape: (8, 256, 256, 1)

# Step 3: Train the model with 2D input
model_lenet.train_on_batch(batch_slices, batch_labels)

## Testing
test_scans = []
test_labels = []
for i in range(test_batches):
    batch = batches_of_paths[i]
    for path in batch:
        scan = nib.load(path)
        data = scan.get_fdata()
        data = resize(data, (256, 256, 180), anti_aliasing=False, preserve_range=True)
 
        details = output_details(path)
        subj_id = details[0]
        description = create_string(details[1])

        label = get_label(labels, subj_id, description)

        if label == 'MCI':
            continue
        elif label == 'AD':
            label = 1
        elif label == 'CN':
            label = 0

        test_scans.append(data)
        test_labels.append(label)

# Convert to arrays
batch_scans = np.array(batch_scans, dtype=np.float32)
batch_labels = np.array(batch_labels)

model_lenel.evaluate(batch_scans, batch_labels)

MPR-R; GradWarp; N3; Scaled
MPR-R; N3; Scaled
MPR-R; N3; Scaled; 2
MPR-R; GradWarp; B1; Correction; N3; Scaled
MPR; GradWarp; B1; Correction; N3; Scaled
MPR; GradWarp; N3; Scaled
MPR; GradWarp; B1; Correction; N3; Scaled
MPR; N3; Scaled
MPR; N3; Scaled; 2
MPR; GradWarp; B1; Correction; N3; Scaled
MPR; GradWarp; B1; Correction; N3; Scaled
MPR; N3; Scaled
MPR; N3; Scaled; 2
MPR; GradWarp; B1; Correction; N3; Scaled
MPR; GradWarp; N3; Scaled
MPR; N3; Scaled
MPR; N3; Scaled; 2
MPR; GradWarp; B1; Correction; N3; Scaled
MPR; GradWarp; B1; Correction; N3; Scaled
MPR; GradWarp; B1; Correction; N3; Scaled
MPR; N3; Scaled
MPR-R; GradWarp; B1; Correction; N3; Scaled
MPR; N3; Scaled; 2
MPR; GradWarp; B1; Correction; N3; Scaled
MPR; GradWarp; B1; Correction; N3; Scaled
MPR; N3; Scaled; 2
MPR; N3; Scaled
MPR; N3; Scaled
MPR; GradWarp; N3; Scaled
MPR; GradWarp; N3; Scaled
MPR; GradWarp; B1; Correction; N3; Scaled
MPR; GradWarp; N3; Scaled
MPR; GradWarp; B1; Correction; N3; Scaled
MPR; GradWarp; B1; C